Competition description

Twitter has become an important communication channel in times of emergency.
The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter (i.e. disaster relief organizations and news agencies).

But, it’s not always clear whether a person’s words are actually announcing a disaster.

Acknowledgments

This dataset was created by the company figure-eight and originally shared on their ‘Data For Everyone’ website here.

Import libraries

In [ ]:
#import libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

Loads train, test, and sample files

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#load files:-
train  = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
sample_sub=pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
train.shape, test.shape

Impute any null values

In [ ]:
train.isnull().sum(), test.isnull().sum()

In [ ]:
# substitue NaN value here with mode

train['location'] = train['location'].fillna(train['location'].mode()[0])
train['keyword'] = train['keyword'].fillna(train['keyword'].mode()[0])

test['location'] = test['location'].fillna(test['location'].mode()[0])
test['keyword'] = test['keyword'].fillna(test['keyword'].mode()[0])

In [ ]:
train.isnull().sum().sum(), test.isnull().sum().sum()

In [ ]:
train.shape, test.shape

Compare tweets in train and test file

In [ ]:
#compare tweets in train and test file
length_train=train['text'].str.len()
length_test=test['text'].str.len()
plt.hist(length_train, bins=20, label="train_tweets")
plt.hist(length_test, bins=20, label="test_tweets")
plt.legend()
plt.show()

Identify percentage of disaster tweets

In [ ]:
train['target'].value_counts()

In [ ]:
percentage_disaster=(train.target.value_counts() / len(train.target)) * 100
percentage_disaster

In [ ]:
label = train.groupby('target')['target'].sum()
label/len(train)*100

In [ ]:
train.groupby('target').text.count().plot.bar(ylim=0)
plt.show()

Preprocessing raw text and getting it ready for machine learning

In [ ]:
# Importing HTMLParser
from html.parser import HTMLParser
html_parser = HTMLParser()

In [ ]:
# Created a new columns i.e. clean_tweet contains the same tweets but cleaned version
train['processed_text'] = train['text'].apply(lambda x: html_parser.unescape(x))
test['processed_text'] = test['text'].apply(lambda x: html_parser.unescape(x))

In [ ]:
# Apostrophe Dictionary
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}
apostrophe_dict

In [ ]:
def lookup_dict(text, dictionary):
    for word in text.split():
        if word.lower() in dictionary:
            if word.lower() in text.split():
                text = text.replace(word, dictionary[word.lower()])
    return text

In [ ]:
train['processed_text'] = train['processed_text'].apply(lambda x: lookup_dict(x,apostrophe_dict))
test['processed_text'] = test['processed_text'].apply(lambda x: lookup_dict(x,apostrophe_dict))

In [ ]:
short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}

In [ ]:
train['processed_text'] = train['processed_text'].apply(lambda x: lookup_dict(x,short_word_dict))
test['processed_text'] = test['processed_text'].apply(lambda x: lookup_dict(x,short_word_dict))

In [ ]:
emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}
emoticon_dict

In [ ]:
train['processed_text'] = train['processed_text'].apply(lambda x: lookup_dict(x,emoticon_dict))
test['processed_text'] = test['processed_text'].apply(lambda x: lookup_dict(x,emoticon_dict))

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
words = stopwords.words("english")

train['processed_text'] = train['text'].apply(lambda x: " ".join([stemmer.stem(i) 
for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

test['processed_text'] = test['text'].apply(lambda x: " ".join([stemmer.stem(i) 
for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

In [ ]:
import string

#make all words lower case
train['processed_text'] = train['processed_text'].str.lower()
test['processed_text'] = test['processed_text'].str.lower()

#Remove punctuation
table = str.maketrans('', '', string.punctuation)
train['processed_text'] = [train['processed_text'][row].translate(table) for row in range(len(train['processed_text']))]
test['processed_text'] = [test['processed_text'][row].translate(table) for row in range(len(test['processed_text']))]

# remove hash tags
train['processed_text'] = train['processed_text'].str.replace("#", " ")
test['processed_text'] = test['processed_text'].str.replace("#", " ")

#remove words less than 1 character
train['processed_text'] = train['processed_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
test['processed_text'] = test['processed_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [ ]:
#put frequent words in a mosiac
freq_words = ' '.join([text for text in train['processed_text']])
from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110, max_words=50).generate(freq_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

Remove rare words

In [ ]:
from collections import Counter
from itertools import chain

# split words into lists
v = train['processed_text'].str.split().tolist() 
# compute global word frequency
c = Counter(chain.from_iterable(v))
# filter, join, and re-assign
train['processed_text'] = [' '.join([j for j in i if c[j] > 1]) for i in v]

# split words into lists
v = test['processed_text'].str.split().tolist() 
# compute global word frequency
c = Counter(chain.from_iterable(v))
# filter, join, and re-assign
test['processed_text'] = [' '.join([j for j in i if c[j] > 1]) for i in v]

In [ ]:
train

In [ ]:
test

Define X, y and X_test

In [ ]:
#define x, y and t_test
y=train.target
X=train['processed_text']
X_test=test['processed_text']

Split train set for training and testing

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.10, random_state=42, shuffle=True)

In [ ]:
X_train.shape, X_val.shape, y_train.shape,y_val.shape

Converting Text to Word Frequency Vectors with TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)
train_tfIdf = vectorizer_tfidf.fit_transform(X_train.values.astype('U'))
val_tfIdf = vectorizer_tfidf.transform(X_val.values.astype('U'))
X_test_tfIdf = vectorizer_tfidf.transform(X_test.values.astype('U'))
print(vectorizer_tfidf.get_feature_names()[:5])

In [ ]:
train_tfIdf.shape,  val_tfIdf.shape, X_test_tfIdf.shape

Define and train the model

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

model=XGBClassifier(max_depth=6, n_estimators=2000)              #MLPClassifier(early_stopping=True)
model.fit(train_tfIdf, y_train)
#create hyper parameters
penalty=['l1', 'l2']
C=np.logspace(0,4,10)
hyperparameters=dict()   #penalty=penalty, C=C)
#create grid search
clf=GridSearchCV(model, hyperparameters, cv=5, verbose=0)
#fit the model
best_model=clf.fit(train_tfIdf, y_train) # training the model
print(clf.score(train_tfIdf, y_train))

In [ ]:
best_model.predict(val_tfIdf)
print(clf.score(val_tfIdf, y_val))

In [ ]:
y_pred=best_model
y_pred = model.predict_proba(val_tfIdf)
y_pred = y_pred >= 0.3
y_pred=y_pred.astype(int)

In [ ]:
prediction = model.predict_proba(val_tfIdf) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 then 1 else 0
prediction_int = prediction_int.astype(np.int)

In [ ]:
# Our Code - For Performance Evaluation
df=pd.DataFrame({'Actual': y_val, 'Predicted':prediction_int})
df

In [ ]:
actual = []
for output in df['Actual']:
    actual.append(output)

predicted = []
for output in df['Predicted']:
    predicted.append(output)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(predicted, actual)
tn, fp, fn, tp = cm.ravel()

print(cm, tn, fp, fn, tp)

In [ ]:
accuracy = (tp + tn)/(tp + tn + fp + fn)
print("accuracy: ", accuracy)

precision = (tp)/(tp + fp)
print("precision: ", precision)

recall = (tp)/(tp + fn)
print("recall: ", recall)

f1 = 2*precision*recall/(precision+recall)
print("f1 score: ", f1)

specificity = (tn)/(tn+fp)
print("specficity: ", specificity)